# 1. 剧情回顾

在上一个[notebook](https://github.com/rikichou/yolo/blob/master/YOLO_face_detection_and_recognition_1--first_sight_of_darknet.ipynb)中，我简要介绍了darknet，它是一个使用C语言实现的深度学习框架，里面提供了YOLO模型的配置文件以及预训练权重，我们可以轻易地通过几行命令就能够实现目标检测。但是假如我们想用自己的数据搭建一个YOLO目标检测模型的话，改怎么做呢？也就是说，我们要利用darknet在我们自己提供的数据上，对YOLO模型进行训练，改怎么做？其实，[darknet](https://pjreddie.com/darknet/yolov2/)官网已经提供了相应的例子来说明如何训练

× 注意：我所说的YOLO其实是YOLOV2

# 2.VOC数据集

严格地说应该叫[Pascal VOC](http://host.robots.ox.ac.uk/pascal/VOC/)数据集，该数据集是一个用于构建图片分类，目标检测，图片分割的数据集。2005--2012年，每年都会举办Pascal VOC挑战赛，在2012年，由于VOC项目的重要成员Mark Everingham去世，所以该比赛就停止了。

该数据集包含了用于目标检测的图片数据以及相应的annotations信息，其中该数据集包含20个类别检测对象。我们将利用2007年和2012年的数据集对YOLO模型进行重新训练。

darknet官网很人性化地提供了VOC数据集的[下载链接](https://pjreddie.com/projects/pascal-voc-dataset-mirror/)，首先进入darknet目录，然后通过以下命令可以直接下载数据集并解压到当前目录：

    wget https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
    wget https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
    wget https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
    
    tar xf VOCtrainval_11-May-2012.tar
    tar xf VOCtrainval_06-Nov-2007.tar
    tar xf VOCtest_06-Nov-2007.tar

## 2.1 了解数据

数据集解压之后，数据的结构大概如下：

--|VOCdevkit  
--|-----VOC2007  
--|----------Annotations  
--|----------ImageSets  
--|---------------Main  
--|----------JPEGImages  
--|----------SegmentationClass  
--|----------SegmentationObject  
--|-----VOC2012  
--|----- .  
--|----- .  
--|----- .  

VOC2007和VOC2012就是我们解压的两个之后的数据集文件夹，他们两个的目录结构完全一致。接下来我们来说说我们需要了解的文件。

Annotations文件夹包含了M个xml文件，M表示图片的数量。这些xml文件里面包含了图片的基本信息，比如图片的大小，以及图片中包含的目标，目标的bounding box，目标的类别等等信息，这些信息在我们训练的时候需要用到。

ImageSets文件夹里面我们只关心Main文件夹，Main文件夹里面都是txt文件，每个txt文件都包含了对应的数据集中每个图片的ID。比如train.txt文件中包含了训练集的图片的ID。

JPEGImages文件夹里面包含了我们真正的图片数据

剩下的两个文件夹是关于图片分割的内容，我们也不关心

# 3 训练

了解了数据之后我们就可以开始按照官网上的教程进行训练流程了。开始训练之前，还需要做一件事情就是训练数据的标签提取

## 3.1 label提取

因为我们是使用darknet来训练YOLO，所以我们需要将数据集的label转换为darknet希望的形式。darknet希望为每一张图片都建立一个对应的txt文件来表示一张图片的信息，该信息的格式如何：

    <object-class> <x> <y> <width> <height>
    
object-class为对象的类别，voc数据集可能有多个类别。  
x为对象（bounding box）的中心点横坐标  
y为对象（bounding box）的中心店的纵坐标  
width为对象（bounding box）的宽度
hight为对象（bounding box）的高度  

其中在TXT文件中，每一行代表一个bounding box的信息，如果一张图片有多个对象，那么对应地就会有多行类似上面的数据

注意：以上的计算都是以图片左上角为坐标原点(0, 0)，图片右下角为坐标(1, 1)，所以以上的取值都在[0,  1]之间

那么清楚了格式之后如何来提取呢？其实，darknet早就帮我我们写好了提取label的脚本了，我已我们只需要下载脚本，运行脚本就好了。

wget https://pjreddie.com/media/files/voc_label.py

python voc_label.py

运行结束之后，在darknet目录下生成了多个TXT文件(2007_train.txt 2007_test.txt 2007_val.txt 2012_train.txt 2012_val.txt)，这些TXT文件包含了对应数据集的中每张图片的绝对路径。

另外，我们会发现在我们每个数据集的目录下（比如darknet/VOCdevkit/VOC2007/labels）多了一个labels文件夹，改文件夹里面包含了许多TXT文件，这些TXT文件就是我们需要的每张图片的label信息。

紧接着，因为我们要将2007_train.txt 2007_val.txt 2012_train.txt 2012_val.txt等文件中表示的图片作为训练集了训练YOLO模型，所以我们把这些TXT文件合并成一个大的TXT文件传输给darknet

cat 2007_train.txt 2007_val.txt 2012_*.txt > train.txt

## 3.2 建立配置文件

在使用darknet训练YOLO模型的时候，我们需要建立配置文件来告诉darknet数据集的信息。这些信息包括

1， 该数据集一共含有多少类对象  
2， 该数据集每一类对象的名字  
3， 包含训练集图片的绝对路径的TXT文件在哪儿  
4， 包含验证集图片的绝对路径的TXT文件在哪儿

darknet已经有了该配置文件cfg/voc.data，我们只需要做适当的修改就好了。

classes= 20  
train  = train.txt  
valid  = 2007_test.txt  
names = data/voc.names  
backup = backup  

因为我的TXT文件都生成在了darknet的目录里面，所以直接写相对路径就好了

## 3.3 下载预训练权重

YOLOV2的论文中，在对YOLO的目标检测模型进行训练之前，将YOLO作为分类模型在ImagNet数据集上进行了预训练，训练的具体参数在这里不提，感兴趣的可以去看看论文。

为什么要这样做呢？我们知道图片数据的基本结构都是很相似的，都是由一些边缘组成基本形状然后再组成抽象的物体，也就是说对于图片的很多特征，特别是底层的特征，我们是可以共享的。既然如此，我就可以先让YOLO在大数据集（imagenet）中学习到一些通用的“知识”（表现为卷积层权重），然后再将这些“知识”作为训练目标检测模型的初始权重，那么这样就可以以这些“知识”为基础进行学习，大大提升了学习的效率，这就是我们所说的迁移学习！

所以我们可以将YOLO模型作为分类模型在imagenet上的1000类的庞大数据集中进行与训练，然后再将这些权重作为初始权重来进行目标检测的学习。当然，我肯定不会去做这个预训练的步骤，感兴趣的朋友可以去论文中获取具体训练的细节。

darknet支持支持在训练的时候载入预训练权重，所以我们只需要下载该权重即可，darknet提供了该权重的下载

wget https://pjreddie.com/media/files/darknet19_448.conv.23

## 3.4 训练

现在一切准备就绪，可以开始训练了，其实训练就一句话

./darknet detector train cfg/voc.data cfg/yolov2-voc.cfg darknet19_448.conv.23

    train:代表我们现在进行的是训练
    cfg/voc.data：是本次训练的配置文件
    cfg/yolov2-voc.cfg：这是YOLOV2模型的配置文件
    darknet19_448.conv.23：这是YOLO模型的预训练权重

我的GPU是GTX 960,有2G显存，对于本次训练，显存倒是够了，但是速度明显不行，训练了大概X个小时。如果是CPU版本的darknet的朋友就不用尝试了，等不起的！

## 4 人脸检测模型的训练

